In [14]:
import cv2
import numpy as np

from scipy import stats
from scipy import fftpack

from dataset import imgs

In [2]:
def shift(im: np.ndarray, w=3):
    assert w % 2 != 0
    assert im.shape[0] >= w
    assert im.shape[1] >= w

    offset = int(w / 2)

    for i in range(0 + offset, im.shape[0] - offset + 1):
        for j in range(0 + offset, im.shape[1] - offset + 1):
            yield im[i - offset : i + offset + 1, j - offset : j + offset + 1]

In [ ]:
def _ncc(s1, s2):
    """Implementation of Normalized Cross Correlation"""

    s1 = (s1 - np.mean(s1)) / (np.std(s1))
    s2 = (s2 - np.mean(s2)) / (np.std(s2))

    return 1 / (s1.shape[0] * s1.shape[1]) * np.sum(np.multiply(s1, s2))


def _ncc(s1, s2):
    """Implementation of Normalized Cross Correlation using CV2"""
    return cv2.matchTemplate(s1, s2, method=cv2.TM_CCORR_NORMED)[0][0]


def vec(s: np.ndarray):
    """Vectorization 2D image -> 1D feature vector"""
    p = np.sum(s, axis=0)
    h = np.sum(s, axis=1)
    mx = np.concatenate((p, h))
    return np.concatenate(
        (
            mx,
            (
                np.max(mx),
                np.min(mx),
                np.var(mx),
                stats.mode(mx).mode,
            ),
        ),
    )


def _ncc(s1, s2):
    """Implementation of Improved Normalized Cross Correlation"""

    v1 = vec(s1)
    v2 = vec(s2)

    v1 = fftpack.dct(v1, norm="ortho")
    v2 = fftpack.dct(v2, norm="ortho")

    v1 = (v1 - np.mean(v1)) / (np.std(v1))
    v2 = (v2 - np.mean(v2)) / (np.std(v2))

    return 1 / (len(v1)) * np.sum(np.multiply(s1, s2))

In [ ]:
def ncc(im1: np.ndarray, im2: np.ndarray, w=3):
    assert im1.shape[0] == im2.shape[0]
    assert im1.shape[1] == im2.shape[1]

    assert w % 2 != 0
    assert im1.shape[0] >= w
    assert im1.shape[1] >= w

    offset = int(w / 2)

    cor = np.zeros(im1.shape)
    for i in range(0 + offset, im1.shape[0] - offset):
        for j in range(0 + offset, im1.shape[1] - offset):
            s1 = im1[i - offset : i + offset + 1, j - offset : j + offset + 1]
            s2 = im2[i - offset : i + offset + 1, j - offset : j + offset + 1]

            cor[i, j] = _ncc(s1, s2)

    return cor


i = 5
w = 3

im1 = imgs[i].im_original.source
im2 = imgs[i].im_inpainted.source

cor = ncc(im1, im2)